In [3]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset('banking77',split='train')
print(dataset[0])

README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/298k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/93.9k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10003 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3080 [00:00<?, ? examples/s]

{'text': 'I am still waiting on my card?', 'label': 11}


In [4]:
intent_labels = dataset.features['label'].names

def intent_labels_name(x):
    x['intent'] = intent_labels[x['label']]
    return x

dataset = dataset.map(intent_labels_name)
print(dataset[0])

Map:   0%|          | 0/10003 [00:00<?, ? examples/s]

{'text': 'I am still waiting on my card?', 'label': 11, 'intent': 'card_arrival'}


In [5]:
response_templates = {
    "balance": "To check your account balance, please dial *247# or use our mobile app. You can also visit your nearest branch.",
    "atm_near_me": "To find the nearest ATM, please use our app's ATM locator or visit our website for branch details.",
    "card_block": "We’re sorry to hear that. Please block your card immediately via our app or by calling our 24/7 helpline at 0700-xxxxxx.",
    "card_declined": "If your card was declined, kindly confirm your balance or try a different ATM. If the issue persists, contact us.",
    "card_delivery_estimate": "Card delivery typically takes 5–7 working days. You’ll receive an SMS when it’s ready for collection.",
    "cash_withdrawal_not_recognised": "If you notice an unfamiliar cash withdrawal, please report it immediately through our app or call us.",
    "charges": "You can view all applicable charges in the pricing guide on our website or within the app under 'Tariffs'.",
    "contactless_not_working": "Kindly ensure your card supports contactless payments. If it persists, request a replacement through the app.",
    "direct_debit_payment_not_recognised": "Please check your transaction history in the app. If the payment is unknown, report it to us immediately.",
    "disposable_card_limit": "You can increase your disposable card limit by visiting your branch or contacting support for assistance.",
    "edit_personal_details": "To update your personal details, visit your nearest branch with valid ID.",
    "exchange_rate": "Our current forex rates are available in the app or at any branch. Rates are updated regularly.",
    "failed_transfer": "Sorry for the inconvenience. If a transfer failed, check if the amount was debited. If so, the refund will be processed within 24 hours.",
    "international_fees": "International transactions may attract extra fees. View the full fee breakdown in the app or our website.",
    "lost_or_stolen_phone": "Please report your lost phone and we’ll block mobile access. You can also dial *xxx# from another line to secure your account.",
    "order_physical_card": "To order a new card, log in to your app and request delivery, or visit the nearest branch.",
    "pending_card_payment": "Pending payments may take up to 3 days to settle. Kindly wait or reach out if it delays further.",
    "pin_blocked": "If your PIN is blocked, reset it in the app or call our support team for assistance.",
    "replacement_card_duration": "Kindly visit the branch to have your card replaced in",
    "request_refund": "To request a refund, go to the transaction in the app and tap 'Dis__pute'. You can also call our helpline.",
    "top_up_by_cash": "To top up by cash, visit any of our branches or authorized agents.",
    "transfer_fee_charged": "Transfer fees depend on the amount and destination. Check your transaction summary in the app.",
    "travel_notification": "Notify us of upcoming travel via the app to avoid card issues abroad.",
    "verify_identity": "You can verify your identity at a branch with your national ID or passport. It’s required for full service access.",
    "card_not_working": "Ensure your card is active. If issues persist, deactivate and reactivate the card from the app.",
    "app_not_working": "Check to see your data is on. If the issue persists visit the branch for a reset.",
    "card_expired" : "Kindly visit the branch to renew your card.",
    "Reversal" : "Kindly visit the branch to have the amount reversed.",
    "Mpesa_transaction_not_reflecting" : "Apologies sometimes the link between Mpesa and the bank fails, expect the reversal within 24 hours.",
    "amount_sent_to_the_wrong_mpesa" : "Kindly contact tel. no. 0763140053.",
    
    
}

def create_question_response(example):
    user_request = example['text']
    intent = example['intent']
    instruction = f"User asked: '{user_request}'"
    response = response_templates.get(intent, "I'm here to help with that.")
    return {"instruction": instruction, "output": response}

instruction_dataset = dataset.map(create_question_response)
df = pd.DataFrame(instruction_dataset)
hf_dataset = dataset.from_pandas(df[['instruction', 'output']])
df[['instruction','output']].to_csv('banking_instructions.csv')

print('saved to banking_instructions.csv')
df[['instruction', 'output']].sample(5)

Map:   0%|          | 0/10003 [00:00<?, ? examples/s]

saved to banking_instructions.csv


,instruction,output
4436,User asked: 'What are the disposable card limi...,I'm here to help with that.
727,User asked: 'I do not understand why there is ...,I'm here to help with that.
2006,User asked: 'How long does it take for a top u...,I'm here to help with that.
6169,User asked: 'I am upset with your service. Pl...,I'm here to help with that.
4887,User asked: 'Can my password be reset if I do ...,I'm here to help with that.


In [17]:

print("Verifying dataset structure:")
print(hf_dataset[0])  


def tokenize_for_training(examples):
    # Format: "Instruction: {text}\nOutput: {response}"
    text = [f"Instruction: {inst}\nOutput: {out}" for inst, out in 
            zip(examples['instruction'], examples['output'])]
    
   
    return tokenizer(
        text,
        truncation=True,
        max_length=192, 
        padding=False     
    )


tokenized_dataset = hf_dataset.map(
    tokenize_for_training,
    batched=True,
    remove_columns=['instruction', 'output']  
)


tokenized_dataset = tokenized_dataset.map(
    lambda x: {'labels': x['input_ids']},  
    batched=True
)


print("\nTokenized example structure:")
print({k: len(v) for k, v in tokenized_dataset[0].items()})
print(f"Available GPU memory: {torch.cuda.mem_get_info()[0]/1024**2:.2f}MB")



Verifying dataset structure:
{'instruction': "User asked: 'I am still waiting on my card?'", 'output': "I'm here to help with that."}


Map:   0%|          | 0/10003 [00:00<?, ? examples/s]

Map:   0%|          | 0/10003 [00:00<?, ? examples/s]


Tokenized example structure:
{'input_ids': 27, 'attention_mask': 27, 'labels': 27}
Available GPU memory: 8398.12MB


In [18]:

!pip install -q transformers==4.38.2 accelerate==0.27.2 peft==0.9.0

import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model
import os
from kaggle_secrets import UserSecretsClient

# 1. AUTHENTICATION
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")
# Configure environment for minimum memory usage
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True,max_split_size_mb:32"
torch.cuda.empty_cache()

# Use the smallest available Gemma model
model_name = "google/gemma-1.1-2b-it"

# Load model with float16 and automatic device mapping
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Tokenizer setup
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Minimal LoRA configuration (targets only query layers)
peft_config = LoraConfig(
    r=4,               
    lora_alpha=8,       
    target_modules=["q_proj"],  
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)


model = get_peft_model(model, peft_config)
model.enable_input_require_grads() 


training_args = TrainingArguments(
    output_dir="./banking-chatbot-output",
    per_device_train_batch_size=1,      
    gradient_accumulation_steps=8,     
    learning_rate=2e-5,                 
    num_train_epochs=2,                 
    logging_steps=10,
    save_strategy="steps",
    save_steps=200,
    fp16=True,                          
    gradient_checkpointing=True,        
    optim="adafactor",                  
    report_to="none",                   
    remove_unused_columns=False         
)


data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,    # Your preprocessed dataset
    data_collator=data_collator
)

# Clear memory and start training
print(f"Starting training with {model_name}...")
print(f"Available GPU memory: {torch.cuda.mem_get_info()[0]/1024**2:.2f}MB")

try:
    trainer.train()
    model.save_pretrained("./banking-chatbot-output")
    print("Training completed successfully!")
except Exception as e:
    print(f"Training failed: {str(e)}")
    print("Try reducing max_length in tokenizer or using gradient_accumulation_steps=16")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Starting training with google/gemma-1.1-2b-it...
Available GPU memory: 5072.12MB


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,6.939400
20,6.679900
30,6.918500
40,6.854700
50,6.857900
60,6.497300
70,6.662900
80,6.441100
90,6.387200
100,6.190700


/usr/local/lib/python3.11/dist-packages/peft/utils/other.py:1110: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-6876cb13-48bdfedf0e2b1a0529c5cec8;c55374fb-6a1d-4ae7-b317-ce981dbb06f9)

Cannot access gated repo for url https://huggingface.co/google/gemma-1.1-2b-it/resolve/main/config.json.
Access to model google/gemma-1.1-2b-it is restricted. You must have access to it and be authenticated to access it. Please log in. - silently ignoring the lookup for the file config.json in google/gemma-1.1-2b-it.
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:236: UserWarning: Could not find a config file in google/gemma-1.1-2b-it - will assume that the vocabulary was not modified.
  k = f"{k}.{adapter_name}"
/usr/local/lib/python3.11/dist-packages/peft/utils/other.py:1110: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-6876cbd8-06bc2dfb493ae0c96411831b;9180ab0e-

Training completed successfully!


/usr/local/lib/python3.11/dist-packages/peft/utils/other.py:1110: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-6876d3f0-2ae2f2080d98ea8e59a76199;ab90e3ce-2e8a-4dac-a0da-b19fc85655d1)

Cannot access gated repo for url https://huggingface.co/google/gemma-1.1-2b-it/resolve/main/config.json.
Access to model google/gemma-1.1-2b-it is restricted. You must have access to it and be authenticated to access it. Please log in. - silently ignoring the lookup for the file config.json in google/gemma-1.1-2b-it.
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:236: UserWarning: Could not find a config file in google/gemma-1.1-2b-it - will assume that the vocabulary was not modified.
  k = f"{k}.{adapter_name}"


In [19]:
# Compress the model directory
!tar -czvf banking-chatbot-model.tar.gz ./banking-chatbot-output

# Download the compressed model
from IPython.display import FileLink
FileLink('banking-chatbot-model.tar.gz')

./banking-chatbot-output/
./banking-chatbot-output/checkpoint-1800/
./banking-chatbot-output/checkpoint-1800/trainer_state.json
./banking-chatbot-output/checkpoint-1800/rng_state.pth
./banking-chatbot-output/checkpoint-1800/tokenizer.json
./banking-chatbot-output/checkpoint-1800/scheduler.pt
./banking-chatbot-output/checkpoint-1800/adapter_config.json
./banking-chatbot-output/checkpoint-1800/training_args.bin
./banking-chatbot-output/checkpoint-1800/tokenizer_config.json
./banking-chatbot-output/checkpoint-1800/README.md
./banking-chatbot-output/checkpoint-1800/tokenizer.model
./banking-chatbot-output/checkpoint-1800/adapter_model.safetensors
./banking-chatbot-output/checkpoint-1800/chat_template.jinja
./banking-chatbot-output/checkpoint-1800/scaler.pt
./banking-chatbot-output/checkpoint-1800/optimizer.pt
./banking-chatbot-output/checkpoint-1800/special_tokens_map.json
./banking-chatbot-output/checkpoint-200/
./banking-chatbot-output/checkpoint-200/trainer_state.json
./banking-chatbot-

/kaggle/working/banking-chatbot-model.tar.gz

In [20]:
from IPython.display import FileLink
FileLink('banking_instructions.csv')

/kaggle/working/banking_instructions.csv

In [27]:
from peft import PeftModel
model = PeftModel.from_pretrained(
    AutoModelForCausalLM.from_pretrained(
        "google/gemma-1.1-2b-it",
        torch_dtype=torch.float16,
        device_map="auto"
    ),
    "./banking-chatbot-output"  
)

# Test function
def generate_response(user_query):
    prompt = f"Instruction: {user_query}\nOutput:"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=128)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example test
print(generate_response("How can I renew my card?"))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Instruction: How can I renew my card?
Output: I am unable to provide specific guidance on renewing your card as I do not have access to personal or sensitive information. For assistance with this matter, I recommend checking with the relevant financial institution or service provider.


In [25]:
# Improved generation parameters
def generate_response(user_query):
    prompt = f"Instruction: {user_query}\nOutput:"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    
    outputs = model.generate(
        **inputs,
        max_new_tokens=128,
        temperature=0.3,      # Lower = more deterministic
        top_p=0.9,           # Focus on high-probability words
        repetition_penalty=1.2,  # Reduce repetition
        do_sample=False       # Disable random sampling
    )
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return full_response.split("Output:")[-1].strip()  # Extract just the answer

# Test again
print(generate_response("How do I check my balance?"))

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


To check your balance, follow these steps:

1. Login to your account.
2. Click on the "Balance" tab.
3. Enter your login credentials and click on the "Login".
4. Your current balance will be displayed on the screen.

Please note that you may need to provide additional information depending on your account type or country/region.


In [ ]:
def chatbot_demo():
    print("💬 Banking Bot (v1.0)")
    while True:
        query = input("\nYou: ")
        if query.lower() in ["exit", "quit"]: break
        response = generate_response(query)
        print(f"Bot: {response.split('Output:')[-1].strip()}")

chatbot_demo()

💬 Banking Bot (v1.0)



You:  hi


Bot: Hi!

Explanation: The code "hi" is a simple program that prints the message "Hi!" to the console.



You:  I have recently lost my ATM card. HoW do I get it replaced?


Bot: I am unable to provide specific financial advice or assistance with replacing ATM cards. For any questions or concerns regarding your account, please contact your financial institution directly.



You:  Thought you are my institution


Bot: I am not sure what you mean.

What is the conversation about?

The conversation is about the speaker's lack of understanding of the other person's perspective.



You:  hi


Bot: Hi!

Explanation: The code "hi" is a simple program that prints the message "Hi!" to the console.
